#### Realizaremos un modelo que identifique si un comentario es positivo o negativo con TensorFlow

1. importamos la librerías que nos permitirá hacer un buen manejo de lso datos 

In [ ]:
# Pandas: Librería para manipulación y análisis de datos estructurados (DataFrames, CSV, etc.)
import pandas as pd  

# NumPy: Manejo eficiente de arreglos numéricos y cálculos matemáticos avanzados
import numpy as np  

# Matplotlib: Creación de gráficos y visualización de datos
import matplotlib.pyplot as plt  

# Deep Translator: Traducción automática de textos utilizando Google Translate
from deep_translator import GoogleTranslator  

2. ahora importaremos la librería que nos permitirá hacer el modelo 

In [ ]:
# TensorFlow: Framework de aprendizaje automático que permite construir y entrenar redes neuronales
import tensorflow as tf 

# Tokenizer: Convierte textos en secuencias numéricas para que el modelo las procese
from tensorflow.keras.preprocessing.text import Tokenizer  

# pad_sequences: Asegura que todas las secuencias tengan la misma longitud agregando ceros si es necesario
from tensorflow.keras.preprocessing.sequence import pad_sequences  

# train_test_split: Divide los datos en conjuntos de entrenamiento y prueba para evaluar el rendimiento del modelo
from sklearn.model_selection import train_test_split  

# load_model: Carga un modelo previamente entrenado desde un archivo
from keras.models import load_model

3. cargamos el DataSet

In [ ]:
# Carga un archivo CSV en un DataFrame de Pandas
df = pd.read_csv('sentiment-analysis.csv')  

# Muestra las primeras cinco filas del DataFrame
df.head()  

4. traducimos todo los datos de la columna Text para trabajar con datos en español 

In [ ]:
# Traduce cada texto automáticamente al español
df['Text'] = df['Text'].apply(lambda x: GoogleTranslator(source='auto', target='es').translate(x))  

# Muestra las primeras cinco filas del DataFrame para verificar la traducción
df.head()  

5. una ves traducido los textos ahora hay que traducir las forma de sentimiento ( parte opcional )

In [ ]:
# Traduce cada etiqueta de sentimiento al español
df['Sentiment'] = df['Sentiment'].apply(lambda x: GoogleTranslator(source='auto', target='es').translate(x))  

# Muestra las primeras cinco filas del DataFrame para verificar la traducción
df.head() 

6. por ultimo traducimos de donde es el usuario 

In [ ]:
# Traduce automáticamente cada ubicación al español
df['Location'] = df['Location'].apply(lambda x: GoogleTranslator(source='auto', target='es').translate(x))  

# Muestra las primeras cinco filas del DataFrame para verificar la traducción
df.head()  

7. exportamos el DataFrame a csv para usarlo para el modelo 

In [ ]:
# Guarda el DataFrame como un archivo CSV sin incluir el índice
df.to_csv('análisis_de_sentimientos.csv', index=False)  

8. una ves traducida y formateada al lenguaje que queremos lo leemos de vuelta para para el modelo

In [ ]:
# Carga el archivo CSV en un DataFrame de Pandas
df = pd.read_csv('análisis_de_sentimientos.csv')  

# Muestra las primeras cinco filas del DataFrame
df.head()  

9. convertimos el tipo de sentimiento en Positivo a 1 Negativo a 0

In [ ]:
# Convierte las etiquetas de sentimiento de texto a valores numéricos (1 para positivo, 0 para negativo)
df['Sentiment'] = df['Sentiment'].map({'Positivo': 1, 'Negativo': 0})  

# Muestra las primeras cinco filas del DataFrame para verificar la conversión
df.head()  

10. debemos hacer que los valores del la columna Text se conviertan a números

In [ ]:
# Define un tokenizador que usa hasta 5000 palabras y asigna "<OOV>" a palabras desconocidas
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")  

# Ajusta el tokenizador usando los textos del dataset
tokenizer.fit_on_texts(df["Text"])  


11. convertimos los valores de Text en una secuencia numérica

In [ ]:
# Convierte los textos en secuencias numéricas
sequences = tokenizer.texts_to_sequences(df['Text'])  

# Muestra las primeras cinco secuencias generadas
sequences[:5] 

In [ ]:
# Ajusta la longitud de todas las secuencias agregando ceros al final cuando sea necesario
padded_sequences = pad_sequences(sequences, padding='post')

# Muestra las primeras cinco secuencias después de aplicar el padding
padded_sequences[:5]

12. División en entrenamiento y prueba

In [ ]:
# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    padded_sequences,  # 🔹 Datos de entrada: secuencias numéricas de los textos procesados
    df["Sentiment"],  # 🔹 Etiquetas de salida: 1 para positivo, 0 para negativo
    test_size=0.2,  # 🔹 Reservamos el 20% de los datos para prueba y el 80% para entrenamiento
    random_state=42  # 🔹 Usamos una semilla aleatoria para asegurar que la división sea reproducible en cada ejecución
)

13. Construcción del Modelo

In [ ]:
# 🔹 Creación del modelo secuencial
model = tf.keras.Sequential([

    #  Capa de Embedding: convierte palabras en vectores numéricos
    tf.keras.layers.Embedding(
        input_dim=5000,  # Define el tamaño del vocabulario (máximo 5000 palabras únicas)
        output_dim=16,  # Especifica la dimensión de los vectores de palabras (cada palabra se representará con 16 valores)
        input_length=padded_sequences.shape[1]  # Define la longitud esperada de las secuencias de entrada
    ),

    #  Global Average Pooling: reduce la dimensión del Embedding promediando los valores
    tf.keras.layers.GlobalAveragePooling1D(),

    #  Capa completamente conectada (oculta) con 16 neuronas y activación ReLU
    tf.keras.layers.Dense(16, activation='relu'),

    #  Capa de salida con una única neurona y activación sigmoide para clasificación binaria
    tf.keras.layers.Dense(1, activation='sigmoid')
])

14. compilamos el modelo y mostramos el resumen 

In [ ]:
# 🔹 Compilamos el modelo antes de entrenarlo
model.compile(
    loss='binary_crossentropy',  #  Función de pérdida utilizada en clasificación binaria; mide la diferencia entre la predicción y la etiqueta real
    optimizer='adam',  #  Optimizador Adam ajusta los pesos del modelo para mejorar su precisión
    metrics=['accuracy']  #  Se usa la métrica de precisión para evaluar el rendimiento durante el entrenamiento
)

# 🔹 Muestra la estructura del modelo, incluyendo el número de capas y parámetros
model.summary()

15. entrenamos el modelo

In [ ]:
# 🔹 Entrenamos el modelo con los datos de entrenamiento
history = model.fit(
    X_train,  #  Datos de entrada para entrenamiento (secuencias de texto numerizadas)
    y_train,  #  Etiquetas de salida para entrenamiento (1 = positivo, 0 = negativo)
    epochs=100,  #  Número de épocas (veces que el modelo verá los datos para aprender)
    validation_data=(X_test, y_test)  # 📌 Datos de validación para evaluar el rendimiento del modelo durante el entrenamiento
)

16. Evaluación del Modelo

In [ ]:
# 🔹 Evaluamos el modelo con los datos de prueba para medir su rendimiento
loss, accuracy = model.evaluate(
    X_test,  #  Conjunto de datos de prueba (secuencias numéricas)
    y_test   #  Etiquetas reales de prueba (1 = positivo, 0 = negativo)
)

# 🔹 Imprimimos la precisión del modelo con dos decimales
print(f"Accuracy: {accuracy:.2f}")  #  Muestra la precisión en porcentaje, indicando qué tan bien predice el modelo

17. hacemos el gráfico de perdida para para saber como le fue al modelo 

#### Este gráfico representa la evolución de la pérdida del modelo durante el entrenamiento y la validación a lo largo de las épocas.

- Eje X (Epochs): Muestra el número de épocas, es decir, la cantidad de veces que el modelo ha procesado el conjunto de datos para ajustar sus parámetros.

- Eje Y (Loss): Representa el valor de la pérdida, que indica qué tan bien (o mal) el modelo está prediciendo los resultados. Un menor valor de pérdida 
significa un mejor ajuste del modelo.

- Línea azul ("Pérdida de Entrenamiento"): Indica cómo disminuye la pérdida en el conjunto de entrenamiento a medida que el modelo aprende.

- Línea roja ("Pérdida de Validación"): Muestra la pérdida en el conjunto de validación, que evalúa el rendimiento del modelo en datos que no ha visto 
durante el entrenamiento.


In [ ]:
# Definimos la lista de épocas basada en la cantidad de iteraciones del entrenamiento
epochs = range(1, len(history.history['loss']) + 1)

# Configuramos el tamaño de la figura para mejorar la visualización
plt.figure(figsize=(30, 8))

# Creamos el primer gráfico: evolución de la pérdida durante el entrenamiento y la validación
plt.subplot(1, 2, 1)  # Dividimos la figura en 1 fila y 2 columnas, seleccionando la primera celda
plt.plot(epochs, history.history['loss'], 'bo-', label='Pérdida de Entrenamiento')  # Trazamos la pérdida del conjunto de entrenamiento
plt.plot(epochs, history.history['val_loss'], 'r*-', label='Pérdida de Validación')  # Trazamos la pérdida del conjunto de validación
plt.xlabel('Epochs')  # Etiqueta del eje X (épocas del entrenamiento)
plt.ylabel('Loss')  # Etiqueta del eje Y (valor de pérdida)
plt.title('Training and Validation Loss')  # Título del gráfico
plt.legend()  # Muestra la leyenda para diferenciar las líneas de entrenamiento y validación
plt.grid()  # Agrega una cuadrícula para mejorar la visualización


18. ahora hacemos el gráfico de precision 

#### Este gráfico representa la evolución de la precisión del modelo durante el entrenamiento y la validación a lo largo de las épocas.

- Eje X (épocas): Indica el número de ciclos completos que el modelo ha realizado con los datos de entrenamiento. Cada época representa una pasada completa por el conjunto de datos.

- Eje Y (nivel de precisión): Muestra la precisión del modelo, es decir, la proporción de predicciones correctas con respecto al total de ejemplos evaluados.

- Línea azul ("Precisión de Entrenamiento"): Representa cómo mejora la precisión del modelo en el conjunto de datos utilizados para el entrenamiento. En general, esta línea debería aumentar con cada época.

- Línea roja ("Precisión de Validación"): Indica la precisión del modelo al evaluar datos nuevos que no ha visto antes. Si esta línea comienza a disminuir mientras la precisión de entrenamiento sigue aumentando, puede ser una señal de sobreajuste.



In [ ]:
# Configuramos el tamaño de la figura para mejorar la visualización de la precisión
plt.figure(figsize=(30, 8))

# Creamos el segundo gráfico: evolución de la precisión durante el entrenamiento y la validación
plt.subplot(1, 2, 2)  # Dividimos la figura en 1 fila y 2 columnas, seleccionando la segunda celda
plt.plot(epochs, history.history['accuracy'], 'bo-', label='Precisión de Entrenamiento')  # Trazamos la precisión del conjunto de entrenamiento
plt.plot(epochs, history.history['val_accuracy'], 'r*-', label='Precisión de Validación')  # Trazamos la precisión del conjunto de validación
plt.xlabel('épocas')  # Etiqueta del eje X (épocas de entrenamiento)
plt.ylabel('nivel de precisión')  # Etiqueta del eje Y (nivel de precisión)
plt.title('Precision de entrenamiento y validaciones ')  # Título del gráfico
plt.legend()  # Muestra la leyenda para diferenciar las líneas de entrenamiento y validación
plt.grid()  # Agrega una cuadrícula para mejorar la lectura de los valores

# Mostramos el gráfico en pantalla
plt.show()

19. hacemos unos textos positivos y negativos para hacer uan predicción de prueba final 

In [ ]:
# Definimos un nuevo texto positivo para evaluar el modelo
new_text_positive = "Me encanta este producto, es increíble y funciona perfectamente."

# Convertimos el texto en una secuencia numérica utilizando el tokenizador entrenado
new_sequence = tokenizer.texts_to_sequences([new_text_positive])

# Aplicamos padding a la secuencia para igualar la longitud esperada por el modelo
new_padded_positive = pad_sequences(
    new_sequence,  # Secuencia numérica generada a partir del texto
    padding="post",  # Agrega ceros al final si la secuencia es más corta que el tamaño esperado
    maxlen=padded_sequences.shape[1]  # Define la longitud máxima basada en las secuencias utilizadas en el entrenamiento
)

In [ ]:
# Definimos un nuevo texto negativo para evaluar el modelo
new_text_negative = "muy malo el producto no me gustó para nada"

# Convertimos el texto en una secuencia numérica utilizando el tokenizador entrenado
new_sequence = tokenizer.texts_to_sequences([new_text_negative])

# Aplicamos padding a la secuencia para igualar la longitud esperada por el modelo
new_padded_negative = pad_sequences(
    new_sequence,  # Secuencia numérica generada a partir del texto
    padding="post",  # Agrega ceros al final si la secuencia es más corta que el tamaño esperado
    maxlen=padded_sequences.shape[1]  # Define la longitud máxima basada en las secuencias utilizadas en el entrenamiento
)

20. hacemos la predicción de los dos comentarios 

In [ ]:
# Realizamos la predicción utilizando el modelo entrenado
prediction = model.predict(new_padded_positive)  # Genera una probabilidad de que el texto tenga un sentimiento positivo

# Interpretamos la predicción: si el resultado es mayor a 0.5, se considera positivo; de lo contrario, negativo
sentimiento = "Positivo" if prediction[0] > 0.5 else "Negativo"

# Imprimimos el texto analizado junto con el sentimiento estimado y la probabilidad calculada
print(f'"{new_text_positive}" → Sentimiento: {sentimiento} ({prediction[0][0]:.2f})')

In [ ]:
# Realizamos la predicción utilizando el modelo entrenado
prediction = model.predict(new_padded_negative)  # Genera una probabilidad de que el texto tenga un sentimiento positivo

# Interpretamos la predicción: si el resultado es mayor a 0.5, se considera positivo; de lo contrario, negativo
sentimiento = "Positivo" if prediction[0] > 0.5 else "Negativo"

# Imprimimos el texto analizado junto con el sentimiento estimado y la probabilidad calculada
print(f'"{new_text_negative}" → Sentimiento: {sentimiento} ({prediction[0][0]:.2f})')

21. exportamos el modelo

In [ ]:
# Guardamos el modelo en formato HDF5 (.h5)
model.save("modelo_sentimientos.h5")  
# Guarda toda la arquitectura, pesos y configuración del modelo

22. cargamos el modelo para probarlo de vuelta asi sabemos si funciona correctamente

In [ ]:
# Cargamos el modelo guardado en formato HDF5
modelo_cargado = load_model("modelo_sentimientos.h5")


In [ ]:

# Realiza una predicción con el modelo cargado con el texto negativo
prediction = modelo_cargado.predict(new_padded_negative)  

sentimiento = "Positivo" if prediction[0] > 0.5 else "Negativo"

print(f'"{new_text_negative}" → Sentimiento: {sentimiento} ({prediction[0][0]:.2f})')

In [ ]:
# realiza una predicción con el modelo cargado con el texto positivo
prediction = modelo_cargado.predict(new_padded_positive)

sentimiento = "Positivo" if prediction[0] > 0.5 else "Negativo"

print(f'"{new_text_positive}" → Sentimiento: {sentimiento} ({prediction[0][0]:.2f})')
